# ファイルからコンテンツを抽出する

このノートブックでは **Azure AI Content Understanding API** を使って、ドキュメント / 音声 / 動画 などマルチモーダルなファイルからセマンティックなコンテンツを抽出する方法を示します。

## 前提条件
1. すでに `CONTENT_UNDERSTANDING_ENDPOINT` と `CONTENT_UNDERSTANDING_KEY` が `.env.dev` に設定されていること。
2. このリポジトリの `data/` フォルダにサンプルファイル (例: `invoice.pdf`, `audio.wav`, `FlightSimulator.mp4`) を配置してください。
3. 必要な Python パッケージをインストールしていない場合は下記セルを実行してください。

In [ ]:
%pip install python-dotenv Pillow requests

## クライアントの初期化 (APIキー方式のみ)
`content_understanding_client.py` は最小実装版です。完全版が必要な場合は公式サンプルから取得して置き換えてください。
環境変数: `CONTENT_UNDERSTANDING_ENDPOINT`, `CONTENT_UNDERSTANDING_KEY` を利用します。

In [2]:
import os, json, logging, uuid, re, importlib, asyncio
from pathlib import Path
from dotenv import load_dotenv, find_dotenv, dotenv_values
from PIL import Image
from io import BytesIO

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# ノートブックから見たリポジトリルートへの相対パス (demos/openai から ../../ でルートへ)
repo_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
env_path = repo_root / '.env.dev'

if env_path.exists():
    # dotenv_values で .env.dev を辞書として読み込む
    parsed = dotenv_values(env_path)
    # 各キーを os.environ に書き込む(空文字や None はスキップ)
    for k, v in parsed.items():
        if v:
            os.environ[k] = v
    print(f'環境変数を読み込みました: {env_path}')
    print(f'{len(parsed)} 個のキーをパースし、os.environ に設定しました。')
else:
    print(f'警告: {env_path} が見つかりません。')


ENDPOINT = os.getenv('CONTENT_UNDERSTANDING_ENDPOINT')
API_KEY = os.getenv('CONTENT_UNDERSTANDING_KEY')
API_VERSION = '2025-05-01-preview'  # 必要に応じて更新

if not ENDPOINT or not API_KEY:
    raise RuntimeError('環境変数 CONTENT_UNDERSTANDING_ENDPOINT / CONTENT_UNDERSTANDING_KEY が設定されていません')

# モジュール再読込 (開いているカーネルが古い定義を持っている場合に備える)
import content_understanding_client
importlib.reload(content_understanding_client)
from content_understanding_client import AzureContentUnderstandingClient, POLL_TIMEOUT_SECONDS

# 非同期メソッドがない古い版を使用中なら動的に追加
if not hasattr(AzureContentUnderstandingClient, 'analyze_and_get_result_async'):
    async def analyze_and_get_result_async(self, analyzer_id: str, file_location: str, timeout_seconds: int = POLL_TIMEOUT_SECONDS, polling_interval_seconds: int = 2):
        resp = await asyncio.to_thread(self.begin_analyze, analyzer_id, file_location)
        return await asyncio.to_thread(self.poll_result, resp, timeout_seconds, polling_interval_seconds)
    AzureContentUnderstandingClient.analyze_and_get_result_async = analyze_and_get_result_async  # type: ignore
    async def begin_analyze_async(self, analyzer_id: str, file_location: str):
        return await asyncio.to_thread(self.begin_analyze, analyzer_id, file_location)
    AzureContentUnderstandingClient.begin_analyze_async = begin_analyze_async  # type: ignore
    async def poll_result_async(self, response, timeout_seconds: int = POLL_TIMEOUT_SECONDS, polling_interval_seconds: int = 2):
        return await asyncio.to_thread(self.poll_result, response, timeout_seconds, polling_interval_seconds)
    AzureContentUnderstandingClient.poll_result_async = poll_result_async  # type: ignore

client = AzureContentUnderstandingClient(endpoint=ENDPOINT, api_version=API_VERSION, subscription_key=API_KEY, x_ms_useragent='azure-ai-content-understanding-demo/content_extraction_jp')

print('Client initialized. Async methods available:', all(hasattr(client, m) for m in ['analyze_and_get_result_async','begin_analyze_async','poll_result_async']))

def save_image(image_id: str, analyze_response):
    raw = client.get_image_from_analyze_operation(analyze_response, image_id)
    if not raw:
        print(f'画像 {image_id} を取得できませんでした')
        return
    img = Image.open(BytesIO(raw))
    Path('.cache').mkdir(exist_ok=True)
    out_path = Path(f'.cache/{image_id}.jpg')
    img.save(out_path, 'JPEG')
    print(f'Saved {out_path}')

環境変数を読み込みました: c:\Users\toohta\Gitrepos\AzureAIServicesDemo\.env.dev
13 個のキーをパースし、os.environ に設定しました。
Client initialized. Async methods available: True


## ドキュメント コンテンツ抽出
`invoice.pdf` を解析し、テキストとレイアウト (テーブル / 図) を Markdown と構造化 JSON で得ます。

In [3]:
DOC_FILE = '../data/invoice.pdf'
ANALYZER_ID = 'prebuilt-documentAnalyzer'

if not Path(DOC_FILE).is_file():
    print(f'サンプルファイルが見つかりません: {DOC_FILE}'); result_doc = None
else:
    resp = client.begin_analyze(ANALYZER_ID, file_location=DOC_FILE)
    result_doc = client.poll_result(resp)
    print(json.dumps(result_doc, indent=2))  # フル出力が長い場合は頭のみ表示

{
  "id": "bf732e18-b7b3-4412-97f3-6febeaad218b",
  "status": "Succeeded",
  "result": {
    "analyzerId": "prebuilt-documentAnalyzer",
    "apiVersion": "2025-05-01-preview",
    "createdAt": "2025-11-13T02:10:10Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "# \u8acb\u6c42\u66f8\n\n\u8acb\u6c42\u65e5: \u5e74\u6708\u65e5\n\n\u8acb\u6c42\u66f8\u756a\u53f7:\n\n\u682a\u5f0f\u4f1a\u793e\n\u5fa1\u4e2d\n\n\u3012\n\n\u96fb\u8a71\u756a\u53f7:\n\u30e1\u30fc\u30eb\u30a2\u30c9\u30ec\u30b9:\n\n\u8acb\u6c42\u91d1\u984d\n\u5186\n\n\n<table>\n<tr>\n<th>\u54c1\u540d</th>\n<th>\u5358\u4fa1</th>\n<th>\u6570\u91cf</th>\n<th>\u91d1\u984d</th>\n</tr>\n<tr>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n<tr>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n<tr>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n<tr>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n<tr>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n</table>\n\n\n<table>\n<tr>\n<td>\u5c0f\u8a08</td>\n

### 抽出された Markdown 表示

In [4]:
if result_doc:
    md = result_doc['result']['contents'][0].get('markdown', '')
    print(md[:4000])

# 請求書

請求日: 年月日

請求書番号:

株式会社
御中

〒

電話番号:
メールアドレス:

請求金額
円


<table>
<tr>
<th>品名</th>
<th>単価</th>
<th>数量</th>
<th>金額</th>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>


<table>
<tr>
<td>小計</td>
<td></td>
</tr>
<tr>
<td>消費税</td>
<td></td>
</tr>
<tr>
<td>合計</td>
<td></td>
</tr>
</table>


<table>
<tr>
<td>振込期日:</td>
<td>年 月 日</td>
</tr>
<tr>
<td>銀行</td>
<td>支店 普通</td>
</tr>
</table>


備考:

<!-- PageBreak -->


## 請求書

請求日:2025年4月30日

請求書番号: S-202504-001

株式会社〇〇御中
☐
☐

アドビデザイン事務所 アドビ花子

〒000-0000
東京都品川区大崎0-0-0 ○○ビル5F
☐
☐

電話番号:00-0000-0000

メールアドレス:○○@example.com
☐
☐

請求金額
330,000円


<table>
<tr>
<th>品名</th>
<th>単価</th>
<th>数量</th>
<th>金額</th>
</tr>
<tr>
<td>2025年4月14日 ロゴデザイン制作</td>
<td>100,000円</td>
<td>1個</td>
<td>100,000円</td>
</tr>
<tr>
<td>2025年4月21日 LPデザイン制作</t

### レイアウト構造 (pages / tables など)

In [5]:
if result_doc:
    content0 = result_doc['result']['contents'][0]
    print(json.dumps(content0.get('tables', []), indent=2)[:2000])

[
  {
    "rowCount": 6,
    "columnCount": 4,
    "cells": [
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 0,
        "rowSpan": 1,
        "columnSpan": 1,
        "content": "\u54c1\u540d",
        "source": "D(1,0.4731,4.0247,3.8115,4.0247,3.8115,4.4494,0.4731,4.4494)",
        "span": {
          "offset": 79,
          "length": 2
        },
        "elements": [
          "/paragraphs/7"
        ]
      },
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 1,
        "rowSpan": 1,
        "columnSpan": 1,
        "content": "\u5358\u4fa1",
        "source": "D(1,3.8115,4.0247,5.033,4.0247,5.033,4.4494,3.8115,4.4494)",
        "span": {
          "offset": 91,
          "length": 2
        },
        "elements": [
          "/paragraphs/8"
        ]
      },
      {
        "kind": "columnHeader",
        "rowIndex": 0,
        "columnIndex": 2,
        "rowSpan": 1,
        "columnSpan": 1,
        "conte

## 音声コンテンツ解析
話者識別 / タイムスタンプ / 信頼度などが含まれます。

In [6]:
AUDIO_FILE = '../data/sampleTokyo.wav'
AUDIO_ANALYZER_ID = 'prebuilt-audioAnalyzer'

if not Path(AUDIO_FILE).is_file():
    print(f'サンプル音声が見つかりません: {AUDIO_FILE}'); audio_result = None
else:
    resp = client.begin_analyze(AUDIO_ANALYZER_ID, file_location=AUDIO_FILE)
    audio_result = client.poll_result(resp)
    print(json.dumps(audio_result, indent=2))

{
  "id": "f62a45f4-db80-4254-9061-60fefda3743a",
  "status": "Succeeded",
  "result": {
    "analyzerId": "prebuilt-audioAnalyzer",
    "apiVersion": "2025-05-01-preview",
    "createdAt": "2025-11-13T02:20:16Z",
    "stringEncoding": "utf8",
    "warnings": [],
    "contents": [
      {
        "markdown": "# Audio: 00:00.000 => 00:48.193\n\nTranscript\n```\nWEBVTT\n\n00:00.040 --> 00:29.360\n<v Speaker 1>\u6d45\u91ce\u670b\u7f8e\u3067\u3059\u3002\u4eca\u65e5\u306e\u6771\u4eac\u682a\u5f0f\u5e02\u5834\u3067\u3001\u65e5\u7d4c\u5e73\u5747\u682a\u4fa1\u306f\u5c0f\u5e45\u7d9a\u4f38\u3068\u306a\u3063\u3066\u3044\u307e\u3059\u3002\u7d42\u5024\u306f\u6628\u65e5\u306b\u6bd4\u307922\u518672\u92ad\u9ad8\u306e11,088\u518658\u92ad\u3067\u3057\u305f\u3002\u6771\u8a3c\u4e00\u90e8\u306e\u5024\u4e0a\u304c\u308a\u9298\u67c4\u6570\u306f1146\u3001\u5bfe\u3057\u3066\u5024\u4e0b\u304c\u308a\u306f368\u3001\u5909\u308f\u3089\u305a\u306f104\u9298\u67c4\u3068\u306a\u3063\u3066\u3044\u307e\u3059\u3002\u3053\u3

In [ ]:
# 解析結果の markdown 内に Unicode エスケープ (\uXXXX) が残っている場合にデコードするユーティリティ
import re

def _decode_unicode_escape_if_needed(text: str) -> str:
    if not isinstance(text, str):
        return text
    # 単純判定: \uNNNN パターンを含む場合のみデコードを試みる
    if re.search(r"\\u[0-9a-fA-F]{4}", text):
        try:
            # unicode_escape で一旦デコード。過度な \n などは保持されるため適切。
            return bytes(text, "utf-8").decode("unicode_escape")
        except Exception:
            return text  # 失敗時はそのまま返す
    return text

# ドキュメント / 音声 / 動画いずれの result オブジェクトにも対応可能

def decode_markdown_in_result(result_json: dict):
    if not isinstance(result_json, dict):
        return result_json
    result_node = result_json.get("result", {})
    contents = result_node.get("contents", [])
    for c in contents:
        if "markdown" in c:
            original = c["markdown"]
            decoded = _decode_unicode_escape_if_needed(original)
            c["markdown_decoded"] = decoded  # 新しいキーに保存 (元を上書きしない)
    return result_json

# 音声結果があればデコードして一部表示
if audio_result:
    audio_result = decode_markdown_in_result(audio_result)
    # 先頭の markdown_decoded を表示
    first = audio_result.get("result", {}).get("contents", [{}])[0].get("markdown_decoded", "(なし)")
    print("--- デコード済み Markdown (先頭) ---\n")
    print(first[:2000])

# 動画結果があれば同様
if 'video_result' in globals() and video_result:
    video_result = decode_markdown_in_result(video_result)
    first_v = video_result.get("result", {}).get("contents", [{}])[0].get("markdown_decoded", "(なし)")
    print("\n--- デコード済み 動画 Markdown (先頭) ---\n")
    print(first_v[:2000])

## 動画コンテンツ解析
ショット / キーフレーム / トランスクリプトなどを取得します。

In [1]:
# 非同期で動画を解析する例 (再試行付き)
import asyncio, json
from pathlib import Path

VIDEO_FILE = '../data/Dev Box Accessibility Up Close Demo.mp4'
VIDEO_ANALYZER_ID = 'prebuilt-videoAnalyzer'

async def run_video_async(max_retries: int = 3):
    if not Path(VIDEO_FILE).is_file():
        print(f'サンプル動画が見つかりません: {VIDEO_FILE}')
        return None
    attempt = 0
    last_err = None
    while attempt < max_retries:
        attempt += 1
        try:
            print(f'Video analyze attempt {attempt}/{max_retries} ...')
            result = await client.analyze_and_get_result_async(VIDEO_ANALYZER_ID, VIDEO_FILE)
            print(json.dumps(result, indent=2)[:3000] + '\n... 省略 ...')
            return result
        except Exception as e:
            last_err = e
            print(f'Attempt {attempt} failed: {e}')
            await asyncio.sleep(2 * attempt)  # バックオフ
    print('解析失敗: 最大リトライ回数に到達しました')
    if last_err:
        print('最後のエラー:', last_err)
    return None

video_result = await run_video_async()

Video analyze attempt 1/3 ...
Attempt 1 failed: name 'client' is not defined
Video analyze attempt 2/3 ...
Attempt 2 failed: name 'client' is not defined
Video analyze attempt 3/3 ...
Attempt 3 failed: name 'client' is not defined
解析失敗: 最大リトライ回数に到達しました
最後のエラー: name 'client' is not defined


### キーフレーム画像保存 (オプション)

In [ ]:
if video_result:

## まとめ
このノートブックでは API キー方式で Content Understanding の基本的な解析 (ドキュメント / 音声 / 動画) を行う最小例を示しました。より高度なトレーニング / 参照ドキュメント生成 / 分類器機能が必要な場合は完全版クライアントを取得してください。